Şerif İnanır, 08.02.2023, 
Written for this topic: https://discuss.pytorch.org/t/how-to-control-tensor-size-inside-of-functional-files-in-c/172027

In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True))
    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels))
    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels//2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)
    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        
        x1 = F.pad(x1, [diffX//2, diffX-diffX//2,
                        diffY//2, diffY-diffY//2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.Sigmoid())
    def forward(self, x):
        return self.conv(x)

In [3]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        factor = 2 if bilinear else 1

        self.inc = DoubleConv(n_channels, 8)
        self.down1 = Down(8, 16)
        self.down2 = Down(16, 32)
        self.down3 = Down(32, 64//factor)
        self.up2 = Up(64, 32//factor, bilinear)        
        self.up3 = Up(32, 16//factor, bilinear)        
        self.up4 = Up(16, 8, bilinear)        
        self.outc = OutConv(8, n_classes)
  
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x = self.up2(x4, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [14]:
def calculate_prediction_durations(first_tensor, second_tensor, loop_count=100, experiment_count=5):
  devs = ["cuda:0", "cpu"]
  tensors = [first_tensor, second_tensor]
  def calculate(model, tensor, device, loop_count):
    durations = []
    outset = None
    for i in range(loop_count):
      outset = time.time()
      model(tensor.to(device))
      durations.append(time.time()-outset)
    return np.mean(durations)
  for  exp in range(1, experiment_count+1):
    for dev in devs:
      #model = UNet(1, 1).to(dev)
      for tensor in tensors:
        model = UNet(1, 1).to(dev)
        print("[EXP {0:4d}]\tDevice: {1:10s}\tInput Size: {2:20s}\tMean Prediction Duration: {3:10f}".format(exp, dev, str(tensor.shape), calculate(model, tensor, dev, loop_count)))
    print("\n")

calculate_prediction_durations(first_tensor=torch.randn(1, 1, 256, 256), second_tensor=torch.randn(1, 1, 128, 128))

[EXP    1]	Device: cuda:0    	Input Size: torch.Size([1, 1, 256, 256])	Mean Prediction Duration:   0.003935
[EXP    1]	Device: cuda:0    	Input Size: torch.Size([1, 1, 128, 128])	Mean Prediction Duration:   0.003833
[EXP    1]	Device: cpu       	Input Size: torch.Size([1, 1, 256, 256])	Mean Prediction Duration:   0.063018
[EXP    1]	Device: cpu       	Input Size: torch.Size([1, 1, 128, 128])	Mean Prediction Duration:   0.020142


[EXP    2]	Device: cuda:0    	Input Size: torch.Size([1, 1, 256, 256])	Mean Prediction Duration:   0.004703
[EXP    2]	Device: cuda:0    	Input Size: torch.Size([1, 1, 128, 128])	Mean Prediction Duration:   0.003693
[EXP    2]	Device: cpu       	Input Size: torch.Size([1, 1, 256, 256])	Mean Prediction Duration:   0.060208
[EXP    2]	Device: cpu       	Input Size: torch.Size([1, 1, 128, 128])	Mean Prediction Duration:   0.014611


[EXP    3]	Device: cuda:0    	Input Size: torch.Size([1, 1, 256, 256])	Mean Prediction Duration:   0.004006
[EXP    3]	Device: cuda: